# Project 1 - Finding the Lane Lines on the Road

In [7]:
#importing useful packages
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Helper Functions

Averaging of the lines was inspired by [naokishibuya](https://github.com/darienmt/CarND-LaneLines-P1/blob/master/P1.ipynb)

In [8]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def canny(img, t1, t2):
    return cv2.Canny(img, t1, t2)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def roi(img, vertices):
    """
    Applies an image mask-- ROI: "Region of interest"
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """      
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return  lines

    ### Averaging lane lines ###

def average_slope_intercept(lines):
    """
    Takes: lines
    Return: left_lane (slope, intercept), right_lane (slope, intercept)
    """
    left_lines    = [] # (slope, intercept)
    left_lines_length  = [] 
    right_lines   = [] # (slope, intercept)
    right_lines_length = [] 
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            slope = (y2-y1)/(x2-x1)     # dy/dx
            intercept = y1 - slope*x1   # Schnittpunkt der y-Achse
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            # arrange lines to the lists by slope (+/-)
            if slope < 0: # y is reversed in image
                left_lines.append((slope, intercept))
                left_lines_length.append((length))
            else:
                right_lines.append((slope, intercept))
                right_lines_length.append((length))
    
    # add more weight to longer lines    
    left_lane  = np.dot(left_lines_length,  left_lines) /np.sum(left_lines_length)  if len(left_lines_length) >0 else None
    right_lane = np.dot(right_lines_length, right_lines)/np.sum(right_lines_length) if len(right_lines_length)>0 else None
    
    return left_lane, right_lane 

def make_line_points(y1, y2, line):
    """
    Convert a line represented in slope and intercept into pixel points
    """
    if line is None:
        return None   
    
    slope, intercept = line
    
    # buildup for cv2.line
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    
    return ((x1, y1), (x2, y2))

def lane_lines(image, lines):
    left_lane, right_lane = average_slope_intercept(lines)
    
    y1 = image.shape[0] # bottom of the image
    y2 = y1*0.6         # slightly lower than the middle

    left_line  = make_line_points(y1, y2, left_lane)
    right_line = make_line_points(y1, y2, right_lane)
    
    return left_line, right_line

    
def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=18):
    # make a separate image to draw lines and combine with the orignal later
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line,  color, thickness)
    # merging the lines to the image 
    return cv2.addWeighted(image, 1.0, line_image, 0.95, 0.0)
             

# Main Function

In [9]:
def process_image(image):
    img = image
    imgc = np.copy(image)
    gray = grayscale(imgc)
    edges = canny(gray, 80, 160)
    blur = gaussian_blur(edges,5)
    
    #defining vertices
    imshape = img.shape
    vertices = np.array([[(imshape[1]*0.04,imshape[0]),(imshape[1]*0.49, imshape[0]*0.56), (imshape[1]*0.51, imshape[0]*0.56), (imshape[1]-imshape[1]*0.04,imshape[0])]], dtype=np.int32)
    
    #masked img
    masked_image = roi(blur, vertices)
    
        # Define the Hough transform parameters
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 150     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 100 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    
    lines = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap)
    
    
    #Draws the lines on the copied input pic
    lines_img = draw_lane_lines(imgc,lane_lines(imgc,lines))
    
    return lines_img

# Testing on a video

In [10]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:40<00:00, 16.81it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 1min 57s, sys: 8.46 s, total: 2min 6s
Wall time: 41.4 s


In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))